# U.S. Medical Insurance Costs

## About this project
This project is part of the Data Analyst Certificate program in CodeAcademy. The goal of the project is to use the Python skills we've been learning and apply them to an open-ended project using real data.

At this point in the program we have not covered numpy or pandas so I will not use those tools to analyze the data for this project.

## Investigating the data
The data for this project is stored in a csv file called insurance.csv. Lets take a look at what is inside the file.

### What fields are in the file?
First lets look at the field names and the first few rows of data

In [11]:
import csv

In [14]:
# The first row will contain the fields in the file
with open('insurance.csv') as insurance_file:
    reader = csv.reader(insurance_file)
    
    for x in range(5):
        print(next(reader))

['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges']
['19', 'female', '27.9', '0', 'yes', 'southwest', '16884.924']
['18', 'male', '33.77', '1', 'no', 'southeast', '1725.5523']
['28', 'male', '33', '3', 'no', 'southeast', '4449.462']
['33', 'male', '22.705', '0', 'no', 'northwest', '21984.47061']


So it looks like we have 7 fields, for information about an individual including:
* Age (*age* - int)
* Sex (*sex* - female/male)
* BMI (*bmi* - float)
* Number of children (*children* - int)
* Whether they are a smoker or not (*smoker* - yes/no)
* The region in the US they come from (*region* - southwest, southeast, northwest, etc.)
* The amount they pay in insurance costs - presumably annually? (*charges* - float)

### How many rows are in the file?
How many rows are in the file.

In [31]:
count = 0
for line in open('insurance.csv').xreadlines(  ): count += 1

print("There are {} lines in the file.".format(count))

AttributeError: '_io.TextIOWrapper' object has no attribute 'xreadlines'

### Are there any non-binary genders in the dataset?
One question I have is whether there are any values for sex beyond "male" and "female".

In [22]:
# Lets get all of the unique values for sex
possible_sexes = []

with open('insurance.csv') as insurance_file:
    csv_reader = csv.DictReader(insurance_file)
        
    for line in csv_reader:
        if line['sex'] not in possible_sexes:
            possible_sexes.append(line['sex'])

print('The values for sex in the data set include:')
for sex in possible_sexes:
    print('- {}'.format(sex))


The values for sex in the data set include:
- female
- male


### What are the possible regions in the dataset?
Another question I have is what the names of all of the regions are.

In [23]:
# Lets get all of the unique values for region
possible_regions = []

with open('insurance.csv') as insurance_file:
    csv_reader = csv.DictReader(insurance_file)
    
    for line in csv_reader:
        if line['region'] not in possible_regions:
            possible_regions.append(line['region'])

print('The values for region in the data set include:')
for region in possible_regions:
    print('- {}'.format(region))

The values for region in the data set include:
- southwest
- southeast
- northwest
- northeast


### Are there any null vales i the dataset?
One last question I have about the data is if there are any null values in the data set that we are going to have to deal with later.

In [28]:
"""
We will start with a simple check to see if a value of None is 
found in any of the fields (meaning that there is a null value)
"""
with open('insurance.csv') as insurance_file:
    reader = csv.reader(insurance_file)
    
    csv_reader = csv.DictReader(insurance_file)
    
    for line in csv_reader:
        for field_value in line:
            if field_value == None:
                print('None found')

## What do we want to find out about the data set?
For this project I would like to find out which factor(s) have the biggest impact on insurance cost.

I'd also like to know if any of the other factors are unduly affected by regionality.

Some other basic questions include:
* What is the average age of patients in the data set?
* Where are the majority of the individuals from?
* What is the average age for someone who has at least one child in the dataset?
* Are there a similar proportion of males to females?
* Is there a similar proportion of people in ages in 10 year buckets?